In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Importing Neccesary libraries
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Handle table-like data and matrices
import numpy as np
import pandas as pd
import math 
# Modelling Algorithms
from sklearn import linear_model
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestRegressor

#preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Modelling Helpers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [ ]:
#Reading the files 
store = pd.read_csv('../input/rossmann-store-sales/store.csv')
#store.head()
store1=store.copy()
df = pd.read_csv('../input/rossmann-store-sales/train.csv', parse_dates = True)
#df.head()
df1=df.copy()
test = pd.read_csv('../input/rossmann-store-sales/test.csv')
#test.head()
test1=test.copy()

In [ ]:
#Preprocessing the given data 
test.fillna(1, inplace=True)
df = df[df["Open"] != 0]
df = df[df["Sales"] > 0]
df['log_sales'] = np.log(df['Sales'])
df = pd.merge(df, store, on='Store')
test = pd.merge(test, store, on='Store')
df.fillna(0,inplace=True)
test.fillna(0,inplace=True)

In [ ]:
#Scaling
df["StateHoliday"] = df["StateHoliday"].map({0: 0, "0": 0, "a": 1, "b": 1, "c": 1})
test["StateHoliday"] = test["StateHoliday"].map({0: 0, "0": 0, "a": 1, "b": 1, "c": 1})
df['StateHoliday'] = df['StateHoliday'].astype(float)
test['StateHoliday'] = test['StateHoliday'].astype(float)

In [ ]:
#Converting the date feature into 3 different features
def extract1(x):
    return int(str(x)[:4])

def extract2(x):
    return int(str(x)[5:7])

def extract3(x):
    return (str(x)[:7])

In [ ]:
df["year"]=df.Date.apply(extract1)
df["month"]=df.Date.apply(extract2)
df["Day"]=df.Date.apply(lambda x: int(str(x)[8:10]))

In [ ]:
test["year"]=test.Date.apply(extract1)
test["month"]=test.Date.apply(extract2)
test["Day"]=test.Date.apply(lambda x: int(str(x)[8:10]))

In [ ]:
#One hot encoding
df = pd.get_dummies(df,columns=['StoreType','Assortment','year'])
test = pd.get_dummies(test,columns=['StoreType','Assortment','year'])
test['year_2013']=0
test['year_2014']=0

In [ ]:
#Dividing the given data set into Input and Output formats
X = df.drop(['Sales','log_sales','Store','Date','Customers','CompetitionOpenSinceYear','Promo2SinceYear','PromoInterval'] , axis = 1)
y = df['log_sales']

In [ ]:
#Dividing the given data set into training and testing splits
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=7)
X_test = test.drop(['Id','Store','Date' ,'CompetitionOpenSinceYear','Promo2SinceYear','PromoInterval'] , axis = 1)

In [ ]:
#Applying Linear Regression model
linreg = linear_model.LinearRegression()
linreg.fit(X_train , y_train)
pred = linreg.predict(X_val)

In [ ]:
rmse_linreg = np.sqrt(mean_squared_error(y_val,pred))
print("RMSE for Linear Regression model is ",rmse_linreg) 

In [ ]:
'''' # Applying Support Vector Regression with Polynomial Kernal
support_1 = make_pipeline(StandardScaler(),SVR(C=1.0, epsilon=0.2,kernal='poly',degree=2))
support_1.fit(X_train, y_train)
pred_svm1=support_1.predict(X_val) '''


In [ ]:
''''rmse_svm_1 = np.sqrt(mean_squared_error(y_val,pred_svm1))
print("RMSE for Support Vector Regression with Polynomial Kernal is", rmse_svm_1) '''

In [ ]:
'''# Applying Support Vector Regression with Gaussian Kernal
support_2 = make_pipeline(StandardScaler(),SVR(C=1.0, epsilon=0.2,kernal='rbf',degree=2))
support_2.fit(X_train, y_train)
pred_svm2=support_2.predict(X_val)  '''


In [ ]:
''''rmse_svm_2 = np.sqrt(mean_squared_error(y_val,pred_svm2))
print("RMSE for Support Vector Regression with Gaussian Kernal is", rmse_svm_2) '''

In [ ]:
# Applying Random Forest Regression 
rand = RandomForestRegressor(n_estimators=100, random_state=0,min_samples_split=5,max_features="auto")
rand.fit(X_train, y_train)
pred = rand.predict(X_val)

In [ ]:
rmse_rand = np.sqrt(mean_squared_error(y_val,pred))
print("RMSE for Random Forest Regression model is ",rmse_rand)

# Random Forest Regression model has given the best RMSE score of 0.13 i.e, 13%  

In [ ]:
#Predicting the sales for the given random data
def final_pred(test1,x):
    test1=test1.append(pd.Series(x,index=['Id','Store','DayOfWeek','Date','Open','Promo','StateHoliday','SchoolHoliday']),ignore_index=True)
    test1.fillna(1, inplace=True)
    test1 = pd.merge(test1, store1, on='Store')
    test1.fillna(0,inplace=True)
    test1["StateHoliday"] = test1["StateHoliday"].map({0: 0, "0": 0, "a": 1, "b": 1, "c": 1})
    test1['StateHoliday'] = test1['StateHoliday'].astype(float)
    test1["year"]=test1.Date.apply(extract1)
    test1["month"]=test1.Date.apply(extract2)
    test1["Day"]=test1.Date.apply(lambda x: int(str(x)[8:10]))
    test1 = pd.get_dummies(test1,columns=['StoreType','Assortment','year'])
    test1['year_2013']=0
    test1['year_2014']=0
    test1 = test1.drop(['Id','Store','Date' ,'CompetitionOpenSinceYear','Promo2SinceYear','PromoInterval'] , axis = 1)
    pred_final = rand.predict(test1)
    return(math.exp(pred_final[41088]))

In [ ]:
#Enter data set with columns 'Id','Store','DayOfWeek','Date','Open','Promo','StateHoliday','SchoolHoliday'
x=[3,7,5,'2015-09-19',1.0,0.0,0,0] 
sales_pred=final_pred(test1,x)
print("Predicted Sales for given data set is",sales_pred)